In [228]:
library('readxl')
library('dplyr')
library('tidyr')

In [241]:
biotech_data <- read_excel("data/biotech.xlsx", sheet = "data")

In [237]:
processed <- biotech_data %>%
  mutate(across(everything(), ~replace_na(.x, 0))) %>%
  mutate(company_name = `Company Name`,
         exchange_ticker = `Exchange:Ticker`,        
         ebitda_margin = ifelse(`Total Revenue [LTM] ($USDmm, Historical rate)` > 0,
                                `EBITDA [LTM] ($USDmm, Historical rate)` / `Total Revenue [LTM] ($USDmm, Historical rate)`, 
                                0),
         adjusted_ebit = `EBIT [LTM] ($USDmm, Historical rate)` + 
                         `R&D Expense [LTM] ($USDmm, Historical rate)` - 
                             (`R&D Expense [LTM - 1] ($USDmm, Historical rate)` + 
                              `R&D Expense [LTM - 2] ($USDmm, Historical rate)` + 
                              `R&D Expense [LTM - 3] ($USDmm, Historical rate)` + 
                              `R&D Expense [LTM - 4] ($USDmm, Historical rate)` + 
                              `R&D Expense [LTM - 5] ($USDmm, Historical rate)`)/5,
         operating_margin = ifelse(`Total Revenue [LTM] ($USDmm, Historical rate)` > 0, 
                                   adjusted_ebit / `Total Revenue [LTM] ($USDmm, Historical rate)`, 
                                   0),
         effective_tax_rate = `Effective Tax Rate [LTM] (%)` / 100,
         capitalized_rd = `R&D Expense [LTM] ($USDmm, Historical rate)` + 
                           0.8 * `R&D Expense [LTM - 1] ($USDmm, Historical rate)` +
                           0.6 * `R&D Expense [LTM - 2] ($USDmm, Historical rate)` +
                           0.4 * `R&D Expense [LTM - 3] ($USDmm, Historical rate)` + 
                           0.2 * `R&D Expense [LTM - 4] ($USDmm, Historical rate)`,
         adjusted_invested_capital = `Total Debt [Latest Quarter] ($USDmm, Historical rate)` +
                                     `Total Equity [Latest Quarter] ($USDmm, Historical rate)` - 
                                     `Cash And Equivalents [Latest Quarter] ($USDmm, Historical rate)` +
                                     capitalized_rd,
         roic = ifelse(adjusted_invested_capital > 0,
                       adjusted_ebit * (1 - effective_tax_rate) / adjusted_invested_capital,
                       0),
         adjusted_net_income = `Net Income [LTM] ($USDmm, Historical rate)` + 
                               `R&D Expense [LTM] ($USDmm, Historical rate)` - 
                               (`R&D Expense [LTM - 1] ($USDmm, Historical rate)` + 
                                `R&D Expense [LTM - 2] ($USDmm, Historical rate)` + 
                                `R&D Expense [LTM - 3] ($USDmm, Historical rate)` + 
                                `R&D Expense [LTM - 4] ($USDmm, Historical rate)` + 
                                `R&D Expense [LTM - 5] ($USDmm, Historical rate)`)/5,
         adjusted_roe = ifelse(`Total Equity [Latest Quarter] ($USDmm, Historical rate)` > 0,
                               adjusted_net_income / `Total Equity [Latest Quarter] ($USDmm, Historical rate)`,
                               0),
         revenue_growth = ifelse(`Total Revenues, 10 Yr CAGR % [LTM] (%)` == 0,
                                 0,
                                 `Total Revenues, 10 Yr CAGR % [LTM] (%)` / 100),
         ebitda_growth = ifelse(`EBITDA, 10 Yr CAGR % [LTM] (%)` == 0,
                                0,
                                `EBITDA, 10 Yr CAGR % [LTM] (%)` / 100),
         capitalized_leases = `Operating Lease Commitment Due +1 [Latest Annual] ($USDmm, Historical rate)` / 1.04 +
                              `Operating Lease Commitment Due +2 [Latest Annual] ($USDmm, Historical rate)` / 1.04 ** 2 +
                              `Operating Lease Commitment Due +3 [Latest Annual] ($USDmm, Historical rate)` / 1.04 ** 3 +
                              `Operating Lease Commitment Due +4 [Latest Annual] ($USDmm, Historical rate)` / 1.04 ** 4 +
                              `Operating Lease Commitment Due +5 [Latest Annual] ($USDmm, Historical rate)` / 1.04 ** 5 +
                              ((`Operating Lease Commitment Due, After 5 Yrs [Latest Annual] ($USDmm, Historical rate)` / 5) * (1 - 1.04**-5)/0.04) / 1.04**5,
         book_value_debt = ifelse(`Total Equity [Latest Quarter] ($USDmm, Historical rate)` > 0,
                                  (capitalized_leases + `Total Debt [Latest Quarter] ($USDmm, Historical rate)`) /
                                  (`Total Equity [Latest Quarter] ($USDmm, Historical rate)` + capitalized_leases + `Total Debt [Latest Quarter] ($USDmm, Historical rate)`),
                                  0),
         market_value_debt = ifelse(`Market Capitalization [Latest] ($USDmm, Historical rate)` > 0,
                                    (capitalized_leases + `Total Debt [Latest Quarter] ($USDmm, Historical rate)`) /
                                    (`Market Capitalization [Latest] ($USDmm, Historical rate)` + capitalized_leases + `Total Debt [Latest Quarter] ($USDmm, Historical rate)`),
                                    0),
         enterprise_value = `Market Capitalization [Latest] ($USDmm, Historical rate)` +
                            `Total Debt [Latest Quarter] ($USDmm, Historical rate)` +
                            capitalized_leases - `Cash And Equivalents [Latest Quarter] ($USDmm, Historical rate)`,
         
         adjusted_total_debt = `Total Debt [Latest Quarter] ($USDmm, Historical rate)` + capitalized_leases,
         pe = ifelse(`Net Income [LTM] ($USDmm, Historical rate)` > 0,
                     `Market Capitalization [Latest] ($USDmm, Historical rate)` / `Net Income [LTM] ($USDmm, Historical rate)`,
                     0),
         non_cash_pe = ifelse(`Net Income [LTM] ($USDmm, Historical rate)` > 0,
                              (`Market Capitalization [Latest] ($USDmm, Historical rate)` - `Cash And Equivalents [Latest Quarter] ($USDmm, Historical rate)`) /
                              (`Net Income [LTM] ($USDmm, Historical rate)` - `Interest and Invest. Income [LTM] ($USDmm, Historical rate)`),
                              0),
         adjusted_pe = ifelse(adjusted_net_income > 0 ,
                              `Market Capitalization [Latest] ($USDmm, Historical rate)` / adjusted_net_income,
                              0),
         ev_sales = ifelse(`Total Revenue [LTM] ($USDmm, Historical rate)` == 0,
                           0,
                           enterprise_value/ `Total Revenue [LTM] ($USDmm, Historical rate)`),
         ev_ebit = ifelse(adjusted_ebit > 0,
                          enterprise_value/adjusted_ebit,
                          0),
         ev_invested_capital = ifelse(adjusted_invested_capital > 0,
                                     enterprise_value/adjusted_invested_capital,
                                     0),
         ev_ebitda = ifelse(`EBITDA [LTM] ($USDmm, Historical rate)` > 0,
                            enterprise_value/`EBITDA [LTM] ($USDmm, Historical rate)`,
                            0),
         ev_adjusted_ebitda = ifelse(`EBITDA [LTM] ($USDmm, Historical rate)` > 0,
                                     enterprise_value/(`EBITDA [LTM] ($USDmm, Historical rate)` + 
                                                       `R&D Expense [LTM] ($USDmm, Historical rate)`),
                                     0),
         turnover_ratio = ifelse(`Daily Value Traded [Latest] ($USDmm, Historical rate)` > 0,
                                 ifelse(`Market Capitalization [Latest] ($USDmm, Historical rate)` > 0 ,
                                     ifelse(`Daily Value Traded [Latest] ($USDmm, Historical rate)` * 250 / 
                                            `Market Capitalization [Latest] ($USDmm, Historical rate)` > 10,
                                            0,
                                        `Daily Value Traded [Latest] ($USDmm, Historical rate)` * 250 / 
                                        `Market Capitalization [Latest] ($USDmm, Historical rate)`),
                                        0),
                                 0),
         only_taxable_income = ifelse(`EBT Incl Unusual Items [LTM] ($USDmm, Historical rate)` > 0,
                                      `EBT Incl Unusual Items [LTM] ($USDmm, Historical rate)`,
                                      0),
         missing_taxable_income = ifelse(`EBT Incl Unusual Items [LTM] ($USDmm, Historical rate)` == 0,
                                         0, 1),
         only_taxes = ifelse(`EBT Incl Unusual Items [LTM] ($USDmm, Historical rate)` > 0,
                             `Income Tax Expense [LTM] ($USDmm, Historical rate)`,
                             0),
         only_net_income = ifelse(`Net Income [LTM] ($USDmm, Historical rate)` > 0,
                                  `Net Income [LTM] ($USDmm, Historical rate)`,
                                  0),
         only_market_cap = ifelse(`Net Income [LTM] ($USDmm, Historical rate)` > 0,
                                  `Market Capitalization [Latest] ($USDmm, Historical rate)`,
                                  0)
        ) %>%
  select("company_name", "exchange_ticker", "ebitda_margin", "operating_margin", "effective_tax_rate", 
         "roic", "adjusted_roe", "revenue_growth", "ebitda_growth", "book_value_debt", "market_value_debt", 
         "enterprise_value", "adjusted_invested_capital", "adjusted_net_income", "adjusted_ebit", 
         "adjusted_total_debt", "pe", "non_cash_pe", "adjusted_pe", "ev_sales", "ev_ebit", 
         "ev_invested_capital", "ev_ebitda", "capitalized_rd", "ev_adjusted_ebitda", "turnover_ratio", 
         "only_taxable_income", "missing_taxable_income", "only_taxes", "only_net_income", "only_market_cap", 
         "capitalized_leases"  ) 

In [238]:
processed

company_name,exchange_ticker,ebitda_margin,operating_margin,effective_tax_rate,roic,adjusted_roe,revenue_growth,ebitda_growth,book_value_debt,...,ev_ebitda,capitalized_rd,ev_adjusted_ebitda,turnover_ratio,only_taxable_income,missing_taxable_income,only_taxes,only_net_income,only_market_cap,capitalized_leases
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Gilead Sciences Inc. (NasdaqGS:GILD),NasdaqGS:GILD,6.704299e-01,6.546573e-01,0.1880,0.516210712,0.80576102,0.3410,0.383,0.44437525,...,9.172784,7420.140,7.833082,1.4797620,14856.0,1,2797.0,12101.0,150429.6,259.650549
Amgen Inc. (NasdaqGS:AMGN),NasdaqGS:AMGN,4.436525e-01,3.470966e-01,0.0765,0.096897524,0.20396462,0.0664,0.068,0.55065202,...,16.991548,13607.800,11.283331,0.8033914,5585.0,1,427.0,5158.0,123383.2,874.565958
Biogen Inc. (NasdaqGS:BIIB),NasdaqGS:BIIB,4.747173e-01,4.348587e-01,0.2520,0.202093302,0.29455891,0.1590,0.253,0.09843983,...,21.896102,5418.580,15.516268,1.9810914,3931.5,1,989.9,2934.8,100853.5,595.562338
Celgene Corporation (NasdaqGS:CELG),NasdaqGS:CELG,4.003051e-01,3.471840e-01,0.1410,0.138024306,0.30131805,0.3510,0.502,0.51998261,...,31.903775,7294.940,18.235734,1.7207972,2327.4,1,327.5,1999.9,95014.1,192.640797
"Regeneron Pharmaceuticals, Inc. (NasdaqGS:REGN)",NasdaqGS:REGN,3.160377e-01,4.034260e-01,0.5510,0.088422552,0.25457263,0.3210,0.455,0.17660154,...,52.128122,3424.800,21.480402,2.0894731,775.7,1,427.7,348.1,46554.8,87.569519
"Alexion Pharmaceuticals, Inc. (NasdaqGS:ALXN)",NasdaqGS:ALXN,4.278999e-01,4.632851e-01,0.2470,0.198550311,0.23496063,0.0000,0.000,0.07061935,...,37.469140,1402.020,24.369219,1.7942207,872.1,1,215.2,656.9,36506.1,86.303761
Vertex Pharmaceuticals Incorporated (NasdaqGS:VRTX),NasdaqGS:VRTX,-9.982771e-01,-1.199070e+00,0.0000,-0.173457689,-0.72344463,0.1890,0.000,0.62332284,...,0.000000,2716.360,0.000000,1.6707857,0.0,1,0.0,0.0,0.0,989.084410
BioMarin Pharmaceutical Inc. (NasdaqGS:BMRN),NasdaqGS:BMRN,-1.348868e-01,-7.400799e-02,0.0000,-0.021823636,-0.02780287,0.4470,0.000,0.30679759,...,0.000000,1236.380,0.000000,0.0000000,0.0,1,0.0,0.0,0.0,18.218133
Pharmacyclics Inc. (NasdaqGS:PCYC),NasdaqGS:PCYC,1.737700e-01,1.838838e-01,0.2970,0.148834832,0.11804149,0.0000,0.000,0.01143255,...,147.547236,649.580,55.123717,2.3126829,122.4,1,36.3,86.1,19544.4,9.589506


In [221]:
library('xlsx')

In [240]:
write.xlsx(processed, 'tmp.xlsx', sheetName = "Sheet1", 
  col.names = TRUE, row.names = TRUE, append = FALSE)

# TODO

1. Make sure prof is ok with querying using column name instead of indexing  
2. Is it wise to use > 0? should we not use if na instead?  